In [1]:
import numpy as np
from astropy.table import Table, vstack
import astropy
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii
import glob
from jagb_func import custom_hist, simple_gaussian, GLOESS
import statistics
from matplotlib import gridspec
from astropy.coordinates import ICRS, Galactic, Distance, Angle, SkyCoord
from astropy import units as u
from astropy.io import fits
from scipy.optimize import leastsq
from scipy.special import erf
from scipy.optimize import curve_fit
from astropy.wcs import WCS
from scipy import stats


### Read in Dust Map

In [2]:
imagename = '/Users/abigaillee/Photometry/M31 PHAT Photometry/Draine Maps/M31_M160_111_SSS_111_Model_All_SurfBr_Mdust.fits'

image_data = fits.getdata(imagename, ext=0)
hdu = fits.open(imagename)[0]
wcs = WCS(hdu.header)
data = hdu.data


In [3]:
def correct_reddening(table,data):
    # use wcs of dust map to transform ra/dec of PHAT stars to X/Y
    x_table, y_table = wcs.all_world2pix(table['ra'],table['dec'],1)
    
    # create bin edges
    binx = np.linspace(0,694,695)-.5
    biny = np.linspace(0,694,695)-.5
    
    # bin number of each star in PHAT table
    ret = stats.binned_statistic_2d(x_table,y_table, None, 'count', bins=[binx,biny])
    
    # eliminate "outer bins"
    ny = len(biny) - 1 # there is one bin less than there are boundaries
    res_binnumber = ret.binnumber - 2 * (ret.binnumber // (ny+2)) - ny - 1 # this is now the bin number for each star in the PHAT able corresponding to the draine maps
    
    table['f110w_cor'] = table['f110w_vega']-data.flatten()[res_binnumber]*.7394/10**5 *.337*.213
    table['f814w_cor'] = table['f814w_vega']-data.flatten()[res_binnumber]*.7394/10**5 *.610*.213
    return table

    

#### Read in PHAT photometry for bricks 18-23

In [ ]:
# apply good star cuts and remove stars with magnitudes of 99.999
t= Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12108-m31-b18_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b18=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b18['Brick']=np.ones(len(b18))*18

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12110-m31-b19_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b19=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b19['Brick']=np.ones(len(b19))*19

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12112-m31-b20_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b20=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b20['Brick']=np.ones(len(b20))*20

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12055-m31-b21_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b21=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b21['Brick']=np.ones(len(b21))*21

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12076-m31-b22_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b22=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b22.remove_rows(b22['field']==8) # remove field 8
b22['Brick']=np.ones(len(b22))*22

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12070-m31-b23_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b23=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b23['Brick']=np.ones(len(b23))*23

t = vstack([b18, b19, b20, b21, b22, b23])


## Correct PHAT photometry for reddening

In [ ]:
t = correct_reddening(t, data)
t['f110w_vega']=t['f110w_cor']
t['f814w_vega']=t['f814w_cor']

## Split Table into 10 regions
These 10 regions were created using the reddening-uncorrected photometry

In [ ]:
# t is all the data
t1 = t[((t['Brick']==18)&(t['field']==18))|((t['Brick']==18)&(t['field']==12))|((t['Brick']==18)&(t['field']==6))
      |((t['Brick']==19)&(t['field']==16))|((t['Brick']==19)&(t['field']==17))|((t['Brick']==18)&(t['field']==17))]

In [ ]:
t2 = t[((t['Brick']==19)&(t['field']==15))|((t['Brick']==18)&(t['field']==11))|((t['Brick']==19)&(t['field']==18))
      |((t['Brick']==18)&(t['field']==5))|((t['Brick']==19)&(t['field']==14))|((t['Brick']==19)&(t['field']==10))
      |((t['Brick']==19)&(t['field']==9))|((t['Brick']==19)&(t['field']==11))]

In [ ]:
t3 = t[((t['Brick']==19)&(t['field']==8))|((t['Brick']==18)&(t['field']==16))|((t['Brick']==19)&(t['field']==13))
      |((t['Brick']==18)&(t['field']==10))|((t['Brick']==19)&(t['field']==12))|((t['Brick']==18)&(t['field']==4))
      |((t['Brick']==19)&(t['field']==7))|((t['Brick']==19)&(t['field']==3))]

In [ ]:
t4 = t[((t['Brick']==19)&(t['field']==4))|((t['Brick']==19)&(t['field']==2))|((t['Brick']==19)&(t['field']==5))
      |((t['Brick']==20)&(t['field']==18))|((t['Brick']==19)&(t['field']==1))|((t['Brick']==20)&(t['field']==12))
      |((t['Brick']==18)&(t['field']==9))|((t['Brick']==18)&(t['field']==3))|((t['Brick']==19)&(t['field']==6))
      |((t['Brick']==18)&(t['field']==15))]

In [ ]:
t5 = t[((t['Brick']==21)&(t['field']==16))|((t['Brick']==21)&(t['field']==15))|((t['Brick']==21)&(t['field']==17))
      |((t['Brick']==20)&(t['field']==6))|((t['Brick']==21)&(t['field']==14))|((t['Brick']==20)&(t['field']==17))
      |((t['Brick']==21)&(t['field']==18))|((t['Brick']==21)&(t['field']==13))|((t['Brick']==20)&(t['field']==11))
      |((t['Brick']==21)&(t['field']==9))]

In [ ]:
t6 = t[((t['Brick']==21)&(t['field']==10))|((t['Brick']==21)&(t['field']==8))|((t['Brick']==20)&(t['field']==5))
      |((t['Brick']==18)&(t['field']==2))|((t['Brick']==21)&(t['field']==11))|((t['Brick']==18)&(t['field']==8))
      |((t['Brick']==21)&(t['field']==7))|((t['Brick']==22)&(t['field']==18))|((t['Brick']==18)&(t['field']==14))
      ]

In [ ]:
t7 = t[((t['Brick']==21)&(t['field']==12))|((t['Brick']==20)&(t['field']==16))|((t['Brick']==21)&(t['field']==3))
      |((t['Brick']==21)&(t['field']==2))|((t['Brick']==22)&(t['field']==12))|((t['Brick']==20)&(t['field']==10))
      |((t['Brick']==21)&(t['field']==1))|((t['Brick']==21)&(t['field']==4))|((t['Brick']==20)&(t['field']==4))
      |((t['Brick']==22)&(t['field']==17))|((t['Brick']==22)&(t['field']==6))]

In [ ]:
t8 = t[((t['Brick']==21)&(t['field']==5))|((t['Brick']==18)&(t['field']==1))|((t['Brick']==22)&(t['field']==11))
      |((t['Brick']==23)&(t['field']==17))|((t['Brick']==23)&(t['field']==18))|((t['Brick']==18)&(t['field']==7))
      |((t['Brick']==23)&(t['field']==16))|((t['Brick']==21)&(t['field']==6))|((t['Brick']==22)&(t['field']==5))
      |((t['Brick']==18)&(t['field']==13))|((t['Brick']==20)&(t['field']==9))|((t['Brick']==20)&(t['field']==3))
      |((t['Brick']==20)&(t['field']==15))]

In [ ]:
t9 = t[((t['Brick']==23)&(t['field']==15))|((t['Brick']==22)&(t['field']==16))|((t['Brick']==22)&(t['field']==10))
      |((t['Brick']==23)&(t['field']==11))|((t['Brick']==23)&(t['field']==10))|((t['Brick']==23)&(t['field']==12))
      |((t['Brick']==22)&(t['field']==4))|((t['Brick']==23)&(t['field']==14))|((t['Brick']==23)&(t['field']==9))
      |((t['Brick']==23)&(t['field']==8))|((t['Brick']==22)&(t['field']==15))|((t['Brick']==20)&(t['field']==2))
      |((t['Brick']==23)&(t['field']==4))|((t['Brick']==20)&(t['field']==8))|((t['Brick']==23)&(t['field']==13))
      |((t['Brick']==22)&(t['field']==9))]

In [ ]:
t10 = t[((t['Brick']==23)&(t['field']==5))|((t['Brick']==23)&(t['field']==3))|((t['Brick']==20)&(t['field']==14))
      |((t['Brick']==22)&(t['field']==3))|((t['Brick']==23)&(t['field']==6))|((t['Brick']==23)&(t['field']==7))
      |((t['Brick']==23)&(t['field']==2))|((t['Brick']==23)&(t['field']==1))|((t['Brick']==22)&(t['field']==14))
      |((t['Brick']==20)&(t['field']==1))|((t['Brick']==22)&(t['field']==2))|((t['Brick']==20)&(t['field']==7))
      |((t['Brick']==20)&(t['field']==13))|((t['Brick']==22)&(t['field']==1))|((t['Brick']==22)&(t['field']==7))
      |((t['Brick']==22)&(t['field']==13))]

### Make tables for JAGB stars

In [ ]:
r1 = t1[((t1['f814w_vega']-t1['f110w_vega'])<2.3)&((t1['f814w_vega']-t1['f110w_vega'])>2)&(t1['f110w_vega']<22)]
r2 = t2[((t2['f814w_vega']-t2['f110w_vega'])<2.3)&((t2['f814w_vega']-t2['f110w_vega'])>2)&(t2['f110w_vega']<22)]
r3 = t3[((t3['f814w_vega']-t3['f110w_vega'])<2.3)&((t3['f814w_vega']-t3['f110w_vega'])>2)&(t3['f110w_vega']<22)]
r4 = t4[((t4['f814w_vega']-t4['f110w_vega'])<2.3)&((t4['f814w_vega']-t4['f110w_vega'])>2)&(t4['f110w_vega']<22)]
r5 = t5[((t5['f814w_vega']-t5['f110w_vega'])<2.3)&((t5['f814w_vega']-t5['f110w_vega'])>2)&(t5['f110w_vega']<22)]
r6 = t6[((t6['f814w_vega']-t6['f110w_vega'])<2.3)&((t6['f814w_vega']-t6['f110w_vega'])>2)&(t6['f110w_vega']<22)]
r7 = t7[((t7['f814w_vega']-t7['f110w_vega'])<2.3)&((t7['f814w_vega']-t7['f110w_vega'])>2)&(t7['f110w_vega']<22)]
r8 = t8[((t8['f814w_vega']-t8['f110w_vega'])<2.3)&((t8['f814w_vega']-t8['f110w_vega'])>2)&(t8['f110w_vega']<22)]
r9 = t9[((t9['f814w_vega']-t9['f110w_vega'])<2.3)&((t9['f814w_vega']-t9['f110w_vega'])>2)&(t9['f110w_vega']<22)]
r10 = t10[((t10['f814w_vega']-t10['f110w_vega'])<2.3)&((t10['f814w_vega']-t10['f110w_vega'])>2)&(t10['f110w_vega']<22)]


